In [1]:
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/ltnga/LawVN-Instructction-Gen/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration constants
WEAVIATE_URL="https://f2l75kjotdg5voubhrvlcg.c0.europe-west3.gcp.weaviate.cloud"
WEAVIATE_API_KEY="eByWVv7b7KT95Zfj0o7TK4EogXOmNphWMmt5"
DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 2000 
CHUNK_OVERLAP = 0 

In [3]:
# Load documents
with open("/home/ltnga/LawVN-Instructction-Gen/src/data/data.json") as f:
    data = json.load(f)
documents = [Document(text=doc) for doc in data]

In [4]:
# Create chunker
chunker = SentenceSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separator=" ",
    include_prev_next_rel=False,  # Keep relationships between chunks
)

In [5]:
# Process nodes
nodes = chunker.get_nodes_from_documents(documents, show_progress=True)
# for node in nodes:
#     node.metadata["original_text"] = node.text  # Store original text
#     node.text = ViTokenizer.tokenize(node.text.lower())  # Tokenize Vietnamese text
#     node.excluded_embed_metadata_keys.append("original_text")
#     node.excluded_llm_metadata_keys.append("original_text")

Parsing nodes: 100%|██████████| 51/51 [00:00<00:00, 469.23it/s]


In [6]:
nodes[38].text

'Ngoài việc bị áp dụng hình thức xử phạt, người điều khiển phương tiện thực hiện hành vi quy định tại điểm c khoản 5, điểm d khoản 7, điểm c khoản 8 Điều này bị trừ điểm giấy phép lái xe được cấp mới nhất trong hệ thống thông tin quản lý giấy phép lái xe 02 điểm.'

In [7]:
nodes[37].text

'Phạt tiền từ 2.000.000 đồng đến 4.000.000 đồng đối với người điều khiển xe mô tô hai bánh có dung tích xi-lanh đến 125 cm3 hoặc có công suất động cơ điện đến 11 kW và các loại xe tương tự xe mô tô thực hiện một trong các hành vi vi phạm sau đây:\na) Không có giấy phép lái xe hoặc sử dụng giấy phép lái xe đã bị trừ hết điểm hoặc sử dụng giấy phép lái xe không do cơ quan có thẩm quyền cấp, giấy phép lái xe bị tẩy xóa, giấy phép lái xe không còn hiệu lực, giấy phép lái xe không phù hợp với loại xe đang điều khiển;\nb) Có giấy phép lái xe quốc tế do các nước tham gia Công ước của Liên hợp quốc về Giao thông đường bộ năm 1968 cấp (trừ giấy phép lái xe quốc tế do Việt Nam cấp) nhưng không mang theo giấy phép lái xe quốc gia phù hợp với loại xe được phép điều khiển;\nc) Sử dụng giấy phép lái xe không hợp lệ (giấy phép lái xe có số phôi ghi ở mặt sau không trùng với số phôi được cấp mới nhất trong hệ thống thông tin quản lý giấy phép lái xe).\n6. Phạt tiền từ 4.000.000 đồng đến 6.000.000 đồng

In [8]:
# Setup vector store
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [9]:
# Create embedding model
embed_model = HuggingFaceEmbedding(
    model_name=MODEL_NAME,
    max_length=8192,
    device=DEVICE,
    trust_remote_code=True
)

In [10]:
# Build index
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    insert_batch_size=32768,  # Optimized batch size
    show_progress=True
)

Generating embeddings: 100%|██████████| 97/97 [00:02<00:00, 40.04it/s]


In [17]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=5,
    alpha=0.5
)

In [18]:
query = "Mức phạt nồng độ cồn 2025 đối với xe mô tô và xe gắn máy"
results = retriever.retrieve(query)
for i, node in enumerate(results):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)


Result 1:

Text: Phạt tiền từ 400.000 đồng đến 600.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:
a) Gây tai nạn giao thông không dừng ngay phương tiện, không giữ nguyên hiện trường, không trợ giúp người bị nạn, không ở lại hiện trường hoặc không đến trình báo ngay với cơ quan công an, Ủy ban nhân dân nơi gần nhất;
b) Điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;
c) Không chấp hành yêu cầu kiểm tra về nồng độ cồn của người thi hành công vụ;
d) Người điều khiển xe đạp máy không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách khi tham gia giao thông trên đường bộ;
đ) Chở người ngồi trên xe đạp máy không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách, trừ trường hợp chở người bệnh đi cấp cứu, trẻ em dưới 